In [ ]:
!apt-get update
!apt-get install -y tesseract-ocr tesseract-ocr-spa poppler-utils
!pip install pytesseract pdf2image pillow PyPDF2

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:7 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,893 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [62.6 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,683 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,720 k

In [ ]:
from pytesseract import image_to_osd

import os
import re
import shutil
import pytesseract
from pdf2image import convert_from_path
from google.colab import files
from PyPDF2 import PdfReader, PdfWriter

In [ ]:
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

Saving image2026-02-06-165213-22.pdf to image2026-02-06-165213-22.pdf


In [ ]:
pages = convert_from_path(
    pdf_path,
    dpi=300
)

In [ ]:
def detectar_rotacion(imagen):
    try:
        osd = pytesseract.image_to_osd(imagen)
        for linea in osd.split("\n"):
            if "Rotate:" in linea:
                return int(linea.split(":")[1].strip())
    except:
        pass
    return 0

def corregir_orientacion(imagen):
    rotacion = detectar_rotacion(imagen)
    if rotacion != 0:
        imagen = imagen.rotate(-rotacion, expand=True)
    return imagen, rotacion


In [ ]:
def extraer_nombre(texto):
    texto = texto.upper().replace("\n", " ")

    patron = re.compile(
        r"([A-ZÁÉÍÓÚÑ\s]+)\s*[\(\[]\s*C\s*C\s*\d+\s*[\)\]]",
        re.UNICODE
    )

    match = patron.search(texto)
    if match:
        return match.group(1).strip()

    return None

In [ ]:
def extraer_fecha(texto):
    texto = texto.upper().replace("\n", " ")

    patron_fecha = re.compile(
        r"FECHA\s*Y\s*HORA\s*[:\-]?\s*(\d{2}/\d{2}/\d{4})",
        re.UNICODE
    )

    match = patron_fecha.search(texto)
    if match:
        return match.group(1)

    return None

In [ ]:
def formatear_fecha(fecha):
    d, m, y = fecha.split("/")
    return f"{y}-{m}-{d}"

In [ ]:
os.makedirs("salida", exist_ok=True)

reader = PdfReader(pdf_path)

for i, (page_img, page_pdf) in enumerate(zip(pages, reader.pages), start=1):

    # 🔄 Corregir orientación
    page_img_corregida, rotacion = corregir_orientacion(page_img)

    # OCR
    texto = pytesseract.image_to_string(
        page_img_corregida,
        lang="spa",
        config="--psm 6"
    )

    nombre = extraer_nombre(texto)
    fecha = extraer_fecha(texto)

    if nombre and fecha:
        nombre_limpio = "_".join(nombre.split())
        fecha_limpia = formatear_fecha(fecha)
        nombre_archivo = f"{nombre_limpio}_{fecha_limpia}.pdf"
    else:
        nombre_archivo = f"SIN_DATOS_{i}.pdf"

    # Crear PDF de una sola página
    writer = PdfWriter()

    if rotacion == 90:
        page_pdf.rotate(-90)
    elif rotacion == 180:
        page_pdf.rotate(-180)
    elif rotacion == 270:
        page_pdf.rotate(-270)

    writer.add_page(page_pdf)

    ruta = os.path.join("salida", nombre_archivo)
    with open(ruta, "wb") as f:
        writer.write(f)

    print(f"✅ Página {i} → {nombre_archivo} (rotación: {rotacion}°)")

✅ Página 1 → NASLY_ADRIANA_FONSECA_CASTILLO_2026-02-25.pdf (rotación: 0°)
✅ Página 2 → VIVIAN_NATALIA_FORERO_AVILA_2026-02-11.pdf (rotación: 0°)
✅ Página 3 → GIOVANNI_MIRANDA_VARGAS_2026-03-03.pdf (rotación: 0°)
✅ Página 4 → GUILLERMO_HERNANDEZ_PINZON_2026-02-26.pdf (rotación: 0°)
✅ Página 5 → WILLMAR_ESNEYDER_ROCHA_GONZALEZ_2026-02-25.pdf (rotación: 0°)
✅ Página 6 → LUIS_GIOVANNY_RODRIGUEZ_CASTELLANOS_2026-02-26.pdf (rotación: 0°)
✅ Página 7 → DEYBER_ANDRES_ROJAS_MENDEZ_2026-02-26.pdf (rotación: 0°)
✅ Página 8 → JOHOXTUM_STEVEN_JIMENEZ_ISAIRIAS_2026-02-26.pdf (rotación: 0°)
✅ Página 9 → CLAUDIA_PATRICIA_RODRIGUEZ_FULA_2026-02-27.pdf (rotación: 0°)
✅ Página 10 → CARLOS_ALBERTO_DIAZ_CORTES_2026-02-27.pdf (rotación: 0°)
✅ Página 11 → LUIS_ALFONSO_GORDILLO_RODRIGUEZ_2026-02-20.pdf (rotación: 0°)
✅ Página 12 → EDUARDO_BELTRAN_GIRALDO_2026-02-19.pdf (rotación: 0°)
✅ Página 13 → HEIDY_JOHANNA_DIAZ_ROA_2026-02-19.pdf (rotación: 0°)
✅ Página 14 → SANDRA_CONSTANZA_AMORTEGUI_RIVEROS_2026-02-20.p

In [ ]:
shutil.make_archive(
    "/content/documentos_finales",
    "zip",
    "/content/salida"
)

files.download("/content/documentos_finales.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os

carpeta = "salida"

for archivo in os.listdir(carpeta):
    ruta = os.path.join(carpeta, archivo)
    if os.path.isfile(ruta):
        os.remove(ruta)

print("✅ Carpeta limpiada")

✅ Carpeta limpiada
